In [1]:
import os
import pandas as pd
import numpy as np

csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

seeds = [42, 123, 456,789,567]

output_dir = 'splits'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for file in csv_files:
    df_original = pd.read_csv(file)
    if 'fitness' in df_original.columns:
        min_value = df_original['fitness'].min()
        max_value = df_original['fitness'].max()
        df_original['fitness'] = (df_original['fitness'] - min_value) / (max_value - min_value)
        
        df_original = df_original.rename(columns={'fitness': 'target'})
    file_prefix = os.path.splitext(file)[0]
    
    
    for n in range(4):
        df = df_original.copy()
        df['set'] = np.where(df['n_mut'] > n, 'test', 'train')
        
        for i, seed in enumerate(seeds):
            np.random.seed(seed)
            
            train_indices = df[df['set'] == 'train'].index
            
            num_validation = max(2, int(len(train_indices) * 0.1))
            
            num_validation = min(len(train_indices), num_validation)
            
            if num_validation > 0:
                validation_indices = np.random.choice(train_indices, size=num_validation, replace=False)
            else:
                validation_indices = []
            if n == 0:
                df['validation'] = ''
            else:
                df['validation'] = ''
                df.loc[validation_indices, 'validation'] = 'true'

            output_file = os.path.join(output_dir, f'{file_prefix}-{n}vsrest-df{i+1}.csv')
            df.to_csv(output_file, index=False)